In [1]:
!pip install mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.5/301.5 kB 2.1 MB/s eta 0:00:00


# TEST API

In [2]:
from mistralai import Mistral

api_key = "Wk4GjfNLY2AGgx32tVlbk40GIjJAjQVe"
model_mistral = "mistral-large-latest"

client = Mistral(api_key=api_key)

chat_response = client.chat.complete(
    model= model_mistral,
    messages = [
        {
            "role": "user",
            "content": "What is the best French cheese?",
        },
    ]
)
print(chat_response.choices[0].message.content)

Choosing the "best" French cheese can be quite subjective, as it depends on personal taste. France is renowned for its wide variety of cheeses, with over 400 different types. Here are a few highly regarded ones:

1. **Camembert de Normandie**: A soft, creamy, and surface-ripened cow's milk cheese from the Normandy region. It's one of the most famous French cheeses worldwide.

2. **Brie de Meaux**: Often referred to as the "King of Cheeses," this soft cheese is produced in the Seine-et-Marne department. It has a smooth, pale interior and a white, bloomy rind.

3. **Roquefort**: A sheep milk blue cheese from the south of France. It has a distinctive tangy and salty flavor and is often crumbled in salads or served with sweet accompaniments.

4. **Comté**: A hard, unpasteurized cow's milk cheese from the Franche-Comté region. It has a strong, slightly sweet flavor and is often used in fondue.

5. **Reblochon**: A soft washed-rind and smear-ripened cheese from the Alps region. It's known fo

# PARTIE BERT

In [3]:
import os

for root, dirs, files in os.walk("/kaggle/input/tuto_bert"):
    print(f"\n📁 Dossier : {root}")
    for file in files:
        print(f"  - {file}")


📁 Dossier : /kaggle/input/tuto_bert

📁 Dossier : /kaggle/input/tuto_bert/keras

📁 Dossier : /kaggle/input/tuto_bert/keras/default

📁 Dossier : /kaggle/input/tuto_bert/keras/default/1

📁 Dossier : /kaggle/input/tuto_bert/keras/default/1/tuto_BERT
  - fingerprint.pb
  - saved_model.pb
  - keras_metadata.pb

📁 Dossier : /kaggle/input/tuto_bert/keras/default/1/tuto_BERT/variables
  - variables.index
  - variables.data-00000-of-00001


In [4]:
import tensorflow as tf

model_path = "/kaggle/input/tuto_bert/keras/default/1/tuto_BERT"

model_bert = tf.saved_model.load(model_path)

2025-04-16 20:22:00.955626: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744834921.273008      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744834921.357244      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1744834941.274408      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1744834941.275058      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability:

In [5]:
from transformers import BertTokenizer
import tensorflow as tf
import numpy as np

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

texts = ["This movie was awesome", "This movie was bad"]

inputs = tokenizer(
    texts,
    return_tensors="tf",
    padding=True,
    truncation=True
)

output = model_bert(inputs)
logits = output["logits"]

probas = tf.nn.softmax(logits, axis=-1)
preds = tf.argmax(probas, axis=1)

print("Classes prédites:", preds.numpy())


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Classes prédites: [1 0]


# COMPARAISON BASIQUE

In [6]:
opinions = [
    "The movie was boring and predictable.",
    "I loved this movie, it was incredible!",
    "The plot was confusing and the characters were underdeveloped.",
    "A masterpiece of cinema, a must-watch.",
    "Disappointing, I expected better."
]

true = [0,1,0,1,0]

**Évaluation de l'API**

In [7]:
import time
pred_mistral=[]
for opinion in opinions:
    chat_response = client.chat.complete(
        model=model_mistral,
        messages=[
            {
                "role": "user",
                "content": f"Evaluate this movie review: '{opinion}' Respond with 0 if the review is negative, 1 if it is positive. Do not respond with anything else than 0 or 1",
            },
        ]
    )

    print(f"Review: {opinion}")
    print(f"Evaluation: {chat_response.choices[0].message.content}\n")
    pred_mistral.append(int(chat_response.choices[0].message.content))
    time.sleep(3)

Review: The movie was boring and predictable.
Evaluation: 0

Review: I loved this movie, it was incredible!
Evaluation: 1

Review: The plot was confusing and the characters were underdeveloped.
Evaluation: 0

Review: A masterpiece of cinema, a must-watch.
Evaluation: 1

Review: Disappointing, I expected better.
Evaluation: 0



In [8]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(true, pred_mistral)

print(f"Accuracy: {accuracy:.2f}")

Accuracy: 1.00


**Évaluation de BERT**

In [9]:
inputs = tokenizer(
    opinions,
    return_tensors="tf",
    padding=True,
    truncation=True
)

output = model_bert(inputs)
logits = output["logits"]

probas = tf.nn.softmax(logits, axis=-1)
pred_bert = tf.argmax(probas, axis=1)
print("Classes prédites:", pred_bert.numpy())

accuracy = accuracy_score(true, pred_bert)

print(f"Accuracy: {accuracy:.2f}")

Classes prédites: [0 1 0 1 0]
Accuracy: 1.00


# COMPARAISON POUSSÉE

In [10]:
opinions = [
    "Best movie for idiots.",
    "The sexual tension between me and the theater exit door watching this.",
    "Google maps please show me the way into Dev Patel's arms",
    "Watched this on the flight to LA and it put me such in a good mood im gonna dance down the I-405 and i dont care if some dude in a lamborghini going 500mph fucking annihilates me",
    "Memory is a strange thing. For example, i remember amy adams' performance as being one of the best of 2016, but the academy seemed to remember differently.",
    "never thought i would leave a film being surprised that cannibalism wasn't involved",
    "Well I just saw the greatest acting performance of my life and it was not by Sandra Hüller or a child actor with the cuntiest bangs I’ve ever seen — it was by a dog so—",
    "Dropping the kids off at Oppenheimer so the adults could watch Barbie",
    "I’m in it with my friends so",
    "the director yelled “cut!” but mia goth heard “cunt!” and just went with it",
    "I just love how fucking disgusting Bruce looks in this. Nasty ass stringy ass translucently pale goth boy in his dank disgusting batcave hunched over his computer with bloodshot eyes and black eye paint scrolling through footage of the previous night and journaling his fucked up thoughts. When he finally emerges from the cave to talk to Alfred and gets disturbed by the natural sunlight and has to put on sunglasses I wanted to get up and cheer. Yeah that’s right put on those sunglasses you little freak. Also he’s definitely a virgin based on how he composes himself around Catwoman. Fucking fantastic movie",
    "this cured my depression and brought it back full force in the span of 2 hours",
    "Sean Baker does for sex workers what Martin Scorsese did for Gangsters",
    "s(he's) bro(ken)",
    "Impressive! A musical with zero (0) good songs! If Selena Gomez gets nominated for an Oscar for this performance I am keying the Academy's cars",
    "gal gadot's acting israelly bad",
    "i gotta get a real job man",
    "I saw someone refer to this as a 'female version of Frankenstein' which is funny because Frankenstein was written by a (brilliant) woman and this was quite obviously written by a man."
]

true = [0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0]

**Évaluation API**

In [11]:
pred_mistral=[]
for opinion in opinions:
    chat_response = client.chat.complete(
        model=model_mistral,
        messages=[
            {
                "role": "user",
                "content": f"Evaluate this movie review: '{opinion}' Respond with 0 if the review is negative, 1 if it is positive. Do not respond with anything else than 0 or 1",
            },
        ]
    )

    print(f"Review: {opinion}")
    print(f"Evaluation: {chat_response.choices[0].message.content}\n")
    pred_mistral.append(int(chat_response.choices[0].message.content))
    time.sleep(3)

Review: Best movie for idiots.
Evaluation: 0

Review: The sexual tension between me and the theater exit door watching this.
Evaluation: 0

Review: Google maps please show me the way into Dev Patel's arms
Evaluation: 1

Review: Watched this on the flight to LA and it put me such in a good mood im gonna dance down the I-405 and i dont care if some dude in a lamborghini going 500mph fucking annihilates me
Evaluation: 1

Review: Memory is a strange thing. For example, i remember amy adams' performance as being one of the best of 2016, but the academy seemed to remember differently.
Evaluation: 1

Review: never thought i would leave a film being surprised that cannibalism wasn't involved
Evaluation: 1

Review: Well I just saw the greatest acting performance of my life and it was not by Sandra Hüller or a child actor with the cuntiest bangs I’ve ever seen — it was by a dog so—
Evaluation: 1

Review: Dropping the kids off at Oppenheimer so the adults could watch Barbie
Evaluation: 1

Review:

In [12]:
accuracy = accuracy_score(true, pred_mistral)

print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.83


**Évaluation BERT**

In [13]:
inputs = tokenizer(
    opinions,
    return_tensors="tf",
    padding=True,
    truncation=True
)

output = model_bert(inputs)
logits = output["logits"]

probas = tf.nn.softmax(logits, axis=-1)
pred_bert = tf.argmax(probas, axis=1)
print("Classes prédites:", pred_bert.numpy())

accuracy = accuracy_score(true, pred_bert)

print(f"Accuracy: {accuracy:.2f}")

Classes prédites: [1 1 1 1 1 0 1 0 1 0 0 1 1 0 1 0 0 0]
Accuracy: 0.56
